**Problem: 1**

$\min e^{x_1 x_2 x_3 x_4 x_5} - \frac{1}{2} (x_1^3 + x_2^3 + 1)^2 \text{ s.t. } x_1^2 + x_2^2 + x_3^3 + x_4^2 + x_5^2 -10 = 0$

$x_2x_3 - 5x_4x_5 = 0$

$x_1^3 + x_2^3 + 1 = 0$

Starting points: $x_0 = \set{-1.71, 1.59, 1.82, -0.763, -0.763}$

Algorithm: Local SQP

$$
\begin{aligned}
&\text{Choose an initial pair $(x_0, \lambda_0)$; set $k \leftarrow 0$.} \\
&\textbf{repeat} \text{ until a convergence test is satisfied} \\
&\quad\quad\quad \text{Evaluate $f_k$, $\nabla f_k$, $\nabla_{xx}^2 \mathcal{L}_k$, $c_k$, and $A_k$;} \\
&\quad\quad\quad \text{Solve $\min\limits_{p} f_k + \nabla f_k^T p + \frac{1}{2} p^T \nabla_{xx}^2 \mathcal{L}_k p$} \\
&\quad\quad\quad \text{s.t. $A_k p + c_k = 0 \text{ if} \min f(x) \text{ s. t. } c(x) = 0$ holds, to obtain $p_k$ and $l_k$;} \\
&\quad\quad\quad \text{Set $x_{k+1} \leftarrow x_k + p_k$ and $\lambda_{k+1} \leftarrow \lambda_k + l_k$;} \\
&\textbf{end repeat}
\end{aligned}
$$

Local SQP in Detail:

$\min f(x) \text{ s. t. } c(x) = 0 \quad$ (18.1a, 18.1b)

$\mathcal{L}(x, \lambda) = f(x) - \lambda^T c(x) \quad$

$A(x)^T = [\nabla c_1(x), \nabla c_2(x), \ldots, \nabla c_m(x)] \quad$ (18.2)

$F(x, \lambda) = \begin{bmatrix} \nabla f(x) - A(x)^T \lambda \\ c(x) \end{bmatrix} = 0 \quad$ (18.3)

$F^\prime(x, \lambda) = \begin{bmatrix} \nabla_{xx}^2 \mathcal{L}(x, \lambda) & -A(x)^T \\ A(x) & 0 \end{bmatrix} \quad$ (18.4) (Jacobian w.r.t. $x$ and $\lambda$)

$\begin{bmatrix} x_{k+1} \\ \lambda_{k+1} \end{bmatrix} = \begin{bmatrix} x_k \\ \lambda_k \end{bmatrix} + \begin{bmatrix} p_k \\ p_\lambda \end{bmatrix} \quad$ (18.5)

$\begin{bmatrix} \nabla_{xx}^2 \mathcal{L}_k & A_k^T \\ A_k & 0 \end{bmatrix} \begin{bmatrix} p_k \\ p_\lambda \end{bmatrix} = \begin{bmatrix} -\nabla f_k + A_k^T \lambda_k \\ -c_k \end{bmatrix} \quad$ (Solve Newton-KKT system) (18.6)

In [1]:
import numpy as np
import sympy as sp
from scipy.linalg import solve
from cvxopt import matrix, solvers

In [2]:
x1, x2, x3, x4, x5 = sp.symbols('x1 x2 x3 x4 x5')
lambda1, lambda2, lambda3 = sp.symbols('lambda1 lambda2 lambda3')

#objective function 
f = sp.exp(x1 * x2 * x3 * x4 * x5) - 0.5 * (x1**3 + x2**3 + 1)**2

#constraints
c1 = x1**2 + x2**2 + x3**3 + x4**2 + x5**2 - 10
c2 = x2 * x3 - 5 * x4 * x5
c3 = x1**3 + x2**3 + 1

#Lagrangian
lagr = f - (lambda1 * c1 + lambda2 * c2 + lambda3 * c3)

#Gradient f_k
grad_f = [sp.diff(f, x) for x in (x1, x2, x3, x4, x5)]

#Jacobian
constraints = [c1, c2, c3]
jacobian_c = sp.Matrix([[sp.diff(c, x) for x in (x1, x2, x3, x4, x5)] for c in constraints])

#Hessian of the Lagrangian \nabla_{xx}^2 L_k
hessian_L = sp.Matrix([[sp.diff(lagr, xi, xj) for xj in (x1, x2, x3, x4, x5)] for xi in (x1, x2, x3, x4, x5)])

grad_f_func = sp.lambdify((x1, x2, x3, x4, x5), grad_f)
jacobian_c_func = sp.lambdify((x1, x2, x3, x4, x5), jacobian_c)
hessian_L_func = sp.lambdify((x1, x2, x3, x4, x5, lambda1, lambda2, lambda3), hessian_L)

#starting points
starting_points = [
    np.array([0, 0, 0, 0, 0]), 
    np.array([1, 0, 3, 0, 0]), 
    np.array([-1.71, 1.59, 1.82, -0.763, -0.763]), 
    np.array([-1.5, 1.2, 1.8, 0.9, -0.815]),
    np.array([-1, 0.9, 1.4, -0.4, -0.9])
]

lambda_k = np.array([0, 0, 0])

tol = 1e-6
max_iter = 10000
epsilon = 1e-10

for i, x_k in enumerate(starting_points):
    print(f"Starting point {x_k}")
    converged = False
    for iteration in range(max_iter):
        #eval. f_k, \nabla f_k, \nabla_{xx}^2 L_k, c_k, A_k
        grad_f_val = np.array(grad_f_func(*x_k))
        jacobian_c_val = np.array(jacobian_c_func(*x_k))
        hessian_L_val = np.array(hessian_L_func(*x_k, *lambda_k))
        f_k_vals = f.subs({x1: x_k[0], x2: x_k[1], x3: x_k[2], x4: x_k[3], x5: x_k[4]})
        c_k_vals = np.array([c.subs({x1: x_k[0], x2: x_k[1], x3: x_k[2], x4: x_k[3], x5: x_k[4]}) for c in constraints], dtype=np.float64)

        print(f"Iteration {iteration + 1}")
        print(f"\nf_k = {f_k_vals}")
        print(f"\nnabla f_k = {grad_f_val}")
        print("\nnabla_xx^2 L_k = ")
        print(hessian_L_val)
        print(f"\nc_k = {c_k_vals}")
        print("\nA_k = ")
        print(jacobian_c_val)

        n_vars = len(x_k)
        n_constraints = len(lambda_k)

        #from page 531####
        #init KKT matrix
        KKT_matrix = np.zeros((n_vars + n_constraints, n_vars + n_constraints))

        #Hessian (top-left)
        KKT_matrix[:n_vars, :n_vars] = hessian_L_val + epsilon * np.eye(n_vars)

        #Jacobian transpose (diagonal)
        KKT_matrix[:n_vars, n_vars:] = -jacobian_c_val.T
        KKT_matrix[n_vars:, :n_vars] = jacobian_c_val
        ##################

        print("\nKKT_matrix = ")
        print(KKT_matrix)
        print("=" * 100)

        #equality on the right of the equation on page 532
        right_eq = np.hstack([-grad_f_val, -c_k_vals])

        #if x0 = [0, 0, 0, 0, 0]
        det = np.linalg.det(KKT_matrix)
        if det == 0:
            print("Determinant of KKT matrix = 0!")
            break

        solution = np.linalg.solve(KKT_matrix, right_eq)
        p_k = solution[:n_vars]
        lambda_k_new = solution[n_vars:]
        x_k_new = x_k + p_k
        
        if np.linalg.norm(p_k) < tol:
            print(f"Converged @ {iteration + 1} iterations")
            converged = True
            break

        x_k = x_k_new
        lambda_k = lambda_k_new

    if converged:
        print(f"x* = {x_k}")
    else:
        print("Solution did not converge!")
    print("=" * 100)

Starting point [0 0 0 0 0]
Iteration 1

f_k = 0.500000000000000

nabla f_k = [0. 0. 0. 0. 0.]

nabla_xx^2 L_k = 
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]

c_k = [-10.   0.   1.]

A_k = 
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]

KKT_matrix = 
[[1.e-10 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 1.e-10 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 1.e-10 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 1.e-10 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 1.e-10 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]
 [0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00 0.e+00]]
Determinant of KKT matrix = 0!
Solution did not converge!
Starting point [1 0 3 0 0]
Iteration 1

f_k = -1.00000000000000

nabla f_k = [-6.  0.  0.  0.  0.]

nabla_xx^2 L_k = 
[[-21.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  

**Problem 2:**

(Rosenbrock function)

$f(x) = 100(x_2 - x_1^2)^2 + (1 - x_1)^2$ with constraints $x_1^2 + x_2^2 \leq 1$ and $x_2 > 0$

Starting points $x_0$ :  $(0.8, 0.6), (1,0), (1,1), (0,0), (-1,1)$ 


In [3]:
x1, x2 = sp.symbols('x1 x2')
lambda1, lambda2 = sp.symbols('lambda1 lambda2')

#f
f = 100 * (x2 - x1**2)**2 + (1 - x1)**2

#contraints
c1 = x1**2 + x2**2 - 1
c2 = -x2

#lagrangian
lagr = f + lambda1 * c1 + lambda2 * c2

#nabla
grad_f = [sp.diff(f, x) for x in (x1, x2)]

#jacobian
constraints = [c1, c2]
jacobian_c = sp.Matrix([[sp.diff(c, x) for x in (x1, x2)] for c in constraints])

#\nabla_{xx}^2 L_k
hessian_L = sp.Matrix([[sp.diff(lagr, xi, xj) for xj in (x1, x2)] for xi in (x1, x2)])

grad_f_func = sp.lambdify((x1, x2), grad_f)
jacobian_c_func = sp.lambdify((x1, x2), jacobian_c)
hessian_L_func = sp.lambdify((x1, x2, lambda1, lambda2), hessian_L)

#starting points
starting_points = [
    np.array([0.8, 0.6]), 
    np.array([1, 0]), 
    np.array([1, 1]), 
    np.array([0, 0]), 
    np.array([-1, 1])
]

lambda_k = np.array([0, 0])

tol = 1e-6
max_iter = 10000
epsilon = 1e-10

for i, x_k in enumerate(starting_points):
    print(f"Starting point {x_k}")
    converged = False
    for iteration in range(max_iter):
        #eval. f_k, \nablaf_k, \nabla_{xx}^2 L_k, c_k, A_k
        grad_f_val = np.array(grad_f_func(*x_k))
        jacobian_c_val = np.array(jacobian_c_func(*x_k))
        hessian_L_val = np.array(hessian_L_func(*x_k, *lambda_k))
        f_k_vals = f.subs({x1: x_k[0], x2: x_k[1]})
        c_k_vals = np.array([c.subs({x1: x_k[0], x2: x_k[1]}) for c in constraints], dtype=np.float64)

        print(f"Iteration {iteration + 1}")
        print(f"\nf_k = {f_k_vals}")
        print(f"\nnabla f_k = {grad_f_val}")
        print("\nnabla_xx^2 L_k = ")
        print(hessian_L_val)
        print(f"\nc_k = {c_k_vals}")
        print("\nA_k = ")
        print(jacobian_c_val)

        n_vars = len(x_k)
        n_constraints = len(lambda_k)
        KKT_matrix = np.zeros((n_vars + n_constraints, n_vars + n_constraints))
        KKT_matrix[:n_vars, :n_vars] = hessian_L_val + epsilon * np.eye(n_vars)
        KKT_matrix[:n_vars, n_vars:] = -jacobian_c_val.T
        KKT_matrix[n_vars:, :n_vars] = jacobian_c_val

        print("\nKKT_matrix = ")
        print(KKT_matrix)
        print("=" * 100)
        right_eq = np.hstack([-grad_f_val, -c_k_vals])
        det = np.linalg.det(KKT_matrix)
        if det == 0:
            print("Determinant of KKT matrix = 0!")
            break

        solution = np.linalg.solve(KKT_matrix, right_eq)
        p_k = solution[:n_vars]
        lambda_k_new = solution[n_vars:]
        x_k_new = x_k + p_k
        
        if np.linalg.norm(p_k) < tol:
            print(f"Converged @ {iteration + 1} iterations")
            converged = True
            break

        x_k = x_k_new
        lambda_k = lambda_k_new

    if converged:
        print(f"x* = {x_k}")
    else:
        print("Solution did not converge!")
    print("=" * 100)

Starting point [0.8 0.6]
Iteration 1

f_k = 0.200000000000001

nabla f_k = [12.4 -8. ]

nabla_xx^2 L_k = 
[[ 530. -320.]
 [-320.  200.]]

c_k = [ 1.11022302e-16 -6.00000000e-01]

A_k = 
[[ 1.6  1.2]
 [ 0.  -1. ]]

KKT_matrix = 
[[ 530.  -320.    -1.6   -0. ]
 [-320.   200.    -1.2    1. ]
 [   1.6    1.2    0.     0. ]
 [   0.    -1.     0.     0. ]]
Iteration 2

f_k = 244.203125000009

nabla f_k = [ 781.75 -312.5 ]

nabla_xx^2 L_k = 
[[2430.625 -500.   ]
 [-500.     753.625]]

c_k = [5.62500000e-01 1.67643677e-14]

A_k = 
[[ 2.50000000e+00 -3.35287353e-14]
 [ 0.00000000e+00 -1.00000000e+00]]

KKT_matrix = 
[[ 2.43062500e+03 -5.00000000e+02 -2.50000000e+00 -0.00000000e+00]
 [-5.00000000e+02  7.53625000e+02  3.35287353e-14  1.00000000e+00]
 [ 2.50000000e+00 -3.35287353e-14  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00  0.00000000e+00  0.00000000e+00]]
Iteration 3

f_k = 110.381914062500

nabla f_k = [ 430.80625 -210.125  ]

nabla_xx^2 L_k = 
[[1450.6375 -410.    ]
 

**Problem 3:**

$f(x) = 150(x_1 x_2)^2 + (0.5 x_1 + 2 x_2 - 2)^2$

Constraints: 

$0 \leq c_1(x) = (x_1 -0.5)^2 + (x_2 - 1)^2 - 5/16$

$0 \leq c_2(x) = (x_1 + 1)^2 + (x_2 - 3/8)^2 - 73/64$

$0 \leq c_3(x) = -(x_1 + 1)^2 - (x_2 - 1)^2 + \sqrt{2}$

Starting points: $x_0 = (0.1, 0.74), (0, 0), (0.5, 0.5), (0, 0.76), (-0.2, 0.8)$

In [9]:
def objective_function(x):
    x1, x2 = x
    return 150 * (x1 * x2) ** 2 + (0.5 * x1 + 2 * x2 - 2) ** 2

def gradient_objective(x):
    x1, x2 = x
    df_dx1 = 300 * x1 * x2 ** 2 + 0.5 * 2 * (0.5 * x1 + 2 * x2 - 2)
    df_dx2 = 300 * x2 * x1 ** 2 + 2 * 2 * (0.5 * x1 + 2 * x2 - 2)
    return np.array([df_dx1, df_dx2])

def hessian_lagrangian(x, lambdas):
    x1, x2 = x
    hess_f = np.array([
        [300 * x2 ** 2 + 0.5**2, 600 * x1 * x2 + 0.5 * 2],
        [600 * x1 * x2 + 0.5 * 2, 300 * x1 ** 2 + 4]
    ])
    return hess_f

def constraints(x):
    x1, x2 = x
    c1 = (x1 - 0.5) ** 2 + (x2 - 1) ** 2 - 5 / 16
    c2 = (x1 + 1) ** 2 + (x2 - 3/8) ** 2 - 73 / 64
    c3 = -(x1 + 1) ** 2 - (x2 - 1) ** 2 + np.sqrt(2)
    return np.array([c1, c2, c3])

def jacobian_constraints(x):
    x1, x2 = x
    dc1_dx1 = 2 * (x1 - 0.5)
    dc1_dx2 = 2 * (x2 - 1)
    dc2_dx1 = 2 * (x1 + 1)
    dc2_dx2 = 2 * (x2 - 3/8)
    dc3_dx1 = -2 * (x1 + 1)
    dc3_dx2 = -2 * (x2 - 1)
    return np.array([
        [dc1_dx1, dc1_dx2],
        [dc2_dx1, dc2_dx2],
        [dc3_dx1, dc3_dx2]])

def sqp_algorithm(x0, lambda0, tol=1e-6, max_iter=10000):
    x_k = np.array(x0)
    lambda_k = np.array(lambda0)

    for k in range(max_iter):
        f_k = objective_function(x_k)
        grad_f_k = gradient_objective(x_k)
        hess_L_k = hessian_lagrangian(x_k, lambda_k)

        c_k = constraints(x_k)
        A_k = jacobian_constraints(x_k)

        # solve to find p_k
        try:
            # [H A.T] [p] = [-grad_f]
            # [A  0 ] [l]   [ -c  ]
            KKT_matrix = np.block([
                [hess_L_k, A_k.T],
                [A_k, np.zeros((len(c_k), len(c_k)))]
            ])
            rhs = np.hstack([-grad_f_k, -c_k])
            solution = solve(KKT_matrix, rhs)
            p_k = solution[:len(x_k)]
            l_k = solution[len(x_k):]
        except np.linalg.LinAlgError:
            print("KKT matrix is singular at iteration", k)
            break

        x_k = x_k + p_k
        lambda_k = lambda_k + l_k

        if np.linalg.norm(p_k) < tol and np.linalg.norm(c_k) < tol:
            print(f"Converged after {k + 1} iterations.")
            break
    else:
        print("Max iters reached without convergence.")

    return x_k, lambda_k

# Starting points
starting_points = [[0.1, 0.74],
                   [0, 0],
                   [0.5, 0.5],
                   [0, 0.76],
                   [-0.2, 0.8]]
# initial lambdas
lambda0 = [0, 0, 0]

for i, x0 in enumerate(starting_points):
    print(f"Starting from point {x0}:")
    solution, lambdas = sqp_algorithm(x0, lambda0)
    print(f"Solution: {solution}")
    print(f"Lagrange multipliers: {lambdas}")
    print("="*50)

Starting from point [0.1, 0.74]:
KKT matrix is singular at iteration 2
Solution: [0.0698554  0.58325633]
Lagrange multipliers: [-3.41131627e+16 -1.90764484e+16 -7.33276394e+15]
Starting from point [0, 0]:
KKT matrix is singular at iteration 0
Solution: [0 0]
Lagrange multipliers: [0 0 0]
Starting from point [0.5, 0.5]:
KKT matrix is singular at iteration 0
Solution: [0.5 0.5]
Lagrange multipliers: [0 0 0]
Starting from point [0, 0.76]:
KKT matrix is singular at iteration 5
Solution: [ 0.66319319 -1.00015295]
Lagrange multipliers: [3.31944071e+16 3.32079329e+16 2.16698969e+16]
Starting from point [-0.2, 0.8]:
KKT matrix is singular at iteration 1
Solution: [0.46579974 0.8       ]
Lagrange multipliers: [ 2.95833335e+17  1.77500001e+17 -8.13541672e+16]


/tmp/ipykernel_3421524/3342502933.py:60: LinAlgWarning: Ill-conditioned matrix (rcond=3.70893e-21): result may not be accurate.
  solution = solve(KKT_matrix, rhs)
/tmp/ipykernel_3421524/3342502933.py:60: LinAlgWarning: Ill-conditioned matrix (rcond=2.30942e-20): result may not be accurate.
  solution = solve(KKT_matrix, rhs)
/tmp/ipykernel_3421524/3342502933.py:60: LinAlgWarning: Ill-conditioned matrix (rcond=8.89051e-21): result may not be accurate.
  solution = solve(KKT_matrix, rhs)
/tmp/ipykernel_3421524/3342502933.py:60: LinAlgWarning: Ill-conditioned matrix (rcond=1.24477e-19): result may not be accurate.
  solution = solve(KKT_matrix, rhs)
/tmp/ipykernel_3421524/3342502933.py:60: LinAlgWarning: Ill-conditioned matrix (rcond=1.38336e-19): result may not be accurate.
  solution = solve(KKT_matrix, rhs)
/tmp/ipykernel_3421524/3342502933.py:60: LinAlgWarning: Ill-conditioned matrix (rcond=7.88637e-20): result may not be accurate.
  solution = solve(KKT_matrix, rhs)
/tmp/ipykernel_3

**Problem 4:**

5 matrices $M$ with $m= 1, 2, 3, 4, 5$ and n = $2, 4, 6, 8, 10$ where the feasible set is $\|x\|_2^2 \coloneqq x_1^2 + x_2^2 + \ldots + x_n^2 \leq 1$

Case 1: $m = 5$ with 2 starting points

Case 2: $m = 10$ via $\tilde{M}$ with 3 starting points (same as the ones used in Project 2)


In [13]:
# using the same problem-set from project 2
solvers.options['show_progress'] = False
def create_qp_matrices(M, y):
    G = M.T @ M
    c = -M.T @ y
    return G, c

class QPProblems:
    def __init__(self):
        self.problems = []
        self._generate_problems()

    def _generate_problems(self):
        LP_problems_2vars = [
            {'A': np.array([[2, 1], [1, 3], [-1, -1]]), 'b': np.array([8, 9, -1]), 'c': np.array([-3, -2]), 'feas_strt_point': np.array([3, 2]), 'non-feas_strt_point': np.array([7, 2])},
            {'A': np.array([[1, 2], [2, 1], [-1, -2]]), 'b': np.array([5, 6, -3]), 'c': np.array([-4, -1]), 'feas_strt_point': np.array([2.3, 1.3]), 'non-feas_strt_point': np.array([1, 3])},
            {'A': np.array([[3, 2], [1, 2], [-2, -1]]), 'b': np.array([12, 8, -4]), 'c': np.array([-2, -3]), 'feas_strt_point': np.array([2, 3]), 'non-feas_strt_point': np.array([2, -0.5])},
            {'A': np.array([[1, 1], [1, -1], [-1, -1]]), 'b': np.array([4, 1, -2]), 'c': np.array([-1, -2]), 'feas_strt_point': np.array([2.5, 1.5]), 'non-feas_strt_point': np.array([0.5, 1])},
            {'A': np.array([[2, 3], [3, 1], [-1, -2]]), 'b': np.array([7, 8, -3]), 'c': np.array([-3, -4]), 'feas_strt_point': np.array([2.4, 0.7]), 'non-feas_strt_point': np.array([0.5, 1.5])}
        ]
        
        for lp_problem in LP_problems_2vars:
            M = lp_problem['A']
            y = lp_problem['b']
            self.problems.append((M, y, [lp_problem['feas_strt_point'], lp_problem['non-feas_strt_point']]))

        LP_problems_10vars = [
            {
                'A': np.array([
                    [1, 1, 0, 1, 0, 0, 1, 1, 1, 1],
                    [2, 1, 0, 1, 2, 1, 0, 1, 2, 1],
                    [0, 2, 1, 2, 1, 3, 1, 2, 1, 0],
                    [1, 2, 0, 3, 1, 2, 0, 1, 4, 1],
                    [2, 1, 0, 1, 3, 1, 1, 2, 1, 2],
                    [1, 1, 0, 1, 1, 2, 0, 0, 1, 0],
                    [0, 0, 3, 3, 0, 1, 2, 0, 0, 1],
                    [0, 1, 1, 1, 2, 1, 1, 1, 1, 1],
                    [1, 1, 0, 0, 1, 0, 2, 1, 0, 0],
                    [0, 3, 1, 0, 1, 2, 1, 0, 0, 1]
                ]),
                'b': np.array([1, 5, 3, 4, 6, 9, 7, 2, 10, 11]),
                'c': np.array([-1, -8, -3, -4, -5, -6, -7, -2, -9, -5]),
                'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
                'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
            },
            {
                'A': np.array([
                    [1, 2, 1, 1, 2, 1, 1, 2, 0, 1],
                    [2, 0, 2, 2, 0, 3, 2, 1, 2, 2],
                    [3, 1, 2, 1, 0, 1, 3, 2, 0, 0],
                    [1, 2, 3, 1, 1, 2, 3, 0, 2, 1],
                    [2, 1, 1, 2, 3, 1, 2, 1, 1, 2],
                    [1, 3, 2, 0, 0, 3, 2, 0, 2, 1],
                    [2, 1, 3, 2, 1, 1, 3, 2, 1, 3],
                    [1, 2, 0, 0, 2, 1, 0, 2, 1, 2],
                    [0, 1, 2, 0, 3, 1, 2, 3, 1, 1],
                    [1, 3, 1, 2, 1, 3, 1, 2, 1, 1]
                ]),
                'b': np.array([12, 14, 16, 18, 20, 22, 24, 26, 28, 30]),
                'c': np.array([-2, -1, -4, -3, -6, -5, -8, -7, -10, -9]),
                'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
                'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
            },
            {
                'A': np.array([
                    [1, 1, 1, 0, 1, 1, 1, 2, 1, 1],
                    [0, 1, 2, 1, 2, 0, 1, 1, 2, 2],
                    [3, 2, 0, 2, 1, 1, 0, 1, 2, 1],
                    [1, 0, 1, 2, 3, 2, 1, 1, 3, 2],
                    [2, 0, 1, 1, 2, 1, 0, 2, 1, 0],
                    [1, 1, 2, 3, 0, 1, 2, 1, 1, 0],
                    [1, 0, 1, 2, 1, 0, 3, 0, 1, 1],
                    [2, 1, 2, 1, 2, 1, 0, 3, 1, 2],
                    [1, 2, 0, 2, 0, 0, 1, 0, 2, 1],
                    [3, 1, 1, 1, 0, 2, 1, 2, 1, 0]
                ]),
                'b': np.array([20, 18, 16, 24, 22, 28, 26, 14, 12, 30]),
                'c': np.array([-3, -1, -5, -2, -4, -6, -8, -7, -9, -10]),
                'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
                'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
            },
            {
                'A': np.array([
                    [2, 1, 2, 1, 2, 1, 2, 0, 2, 1],
                    [1, 2, 1, 3, 0, 2, 1, 1, 1, 2],
                    [1, 2, 1, 0, 4, 1, 2, 0, 0, 1],
                    [1, 0, 2, 1, 2, 2, 1, 1, 0, 2],
                    [2, 1, 3, 0, 1, 3, 2, 1, 0, 1],
                    [1, 1, 2, 1, 3, 2, 1, 1, 0, 1],
                    [2, 3, 1, 0, 3, 2, 1, 2, 0, 2],
                    [1, 1, 3, 2, 1, 3, 2, 1, 1, 2],
                    [0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
                    [1, 2, 3, 1, 0, 1, 2, 0, 1, 1]
                ]),
                'b': np.array([14, 16, 18, 20, 22, 24, 36, 40, 44, 48]),
                'c': np.array([-2, -3, -1, -4, -5, -6, -7, -8, -9, -10]),
                'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
                'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
            },
            {
                'A': np.array([
                    [0, 2, 1, 1, 2, 0, 1, 0, 2, 1],
                    [2, 1, 0, 2, 1, 0, 0, 1, 0, 1],
                    [0, 1, 2, 1, 0, 1, 2, 1, 1, 2],
                    [1, 0, 1, 2, 0, 2, 1, 1, 2, 1],
                    [2, 1, 0, 2, 1, 0, 1, 1, 0, 2],
                    [1, 2, 1, 3, 1, 1, 2, 1, 0, 2],
                    [0, 1, 0, 1, 1, 2, 1, 3, 2, 1],
                    [1, 3, 1, 1, 0, 0, 1, 2, 0, 2],
                    [2, 1, 2, 0, 1, 0, 2, 1, 0, 1],
                    [0, 1, 1, 2, 1, 1, 1, 3, 1, 2]
                ]),
                'b': np.array([15, 27, 19, 31, 21, 23, 17, 25, 29, 13]),
                'c': np.array([-1, -24, -2, -15, -3, -16, -7, -28, -9, -10]),
                'feas_strt_point': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
                'non-feas_strt_point': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
            }]
        
        for lp_problem in LP_problems_10vars:
            M = lp_problem['A']
            y = lp_problem['b']
            self.problems.append((M, y, [lp_problem['feas_strt_point'], lp_problem['non-feas_strt_point']]))

    def get_problem(self, index):
        M, y, start_points = self.problems[index]
        G, c = create_qp_matrices(M, y)
        A = np.vstack([np.eye(len(c)), -np.eye(len(c))])
        b = np.ones(2 * len(c))
        return G, c, A, b, start_points, M, y

def sqp_solver(G, c, A, b, x0, tol=1e-6, max_iter=100, reg_param=1e-6):
    n = x0.shape[0]
    x_k = x0
    lambda_k = np.zeros(A.shape[0])
    function_values = []
    
    def objective_value(x):
        return 0.5 * x.T @ G @ x + c.T @ x
    
    #inequal. constraint for norm
    G_norm = np.vstack([np.eye(n), -np.eye(n)])
    h_norm = np.hstack([np.ones(n), np.ones(n)])
    
    for k in range(max_iter):
        G_reg = G + reg_param * np.eye(n)
        G_cvx = matrix(G_reg)
        c_cvx = matrix(G @ x_k + c, (len(c), 1), 'd')
        G_combined = matrix(np.vstack([A, G_norm]))
        h_combined = matrix(np.hstack([b, h_norm]))

        sol = solvers.qp(G_cvx, c_cvx, G_combined, h_combined)
        p_k = np.array(sol['x']).flatten()

        x_k_new = x_k + p_k

        function_values.append(objective_value(x_k_new))
        lambda_k_new = np.array(sol['z']).flatten()

        if np.linalg.norm(p_k) < tol:
            print(f"Converged after {k + 1} iters.")
            return x_k_new, lambda_k_new, function_values

        x_k = x_k_new
        lambda_k = lambda_k_new
    
    print("Max iters reached without convergence!")
    return x_k, lambda_k, function_values

qp_problems = QPProblems()

for i in range(len(qp_problems.problems)):
    G, c, A, b, start_points, M, y = qp_problems.get_problem(i)
    for j, x0 in enumerate(start_points):
        print(f"Problem {i+1} with SQP from starting_point {j+1}:")
        print("Initial guess x0:")
        print(x0)
        solution, lagrange_multipliers, function_values = sqp_solver(G, c, A, b, x0)
        print(f"Problem {i+1}, solution for starting_point {j+1}: {solution}")
        print("="*50)

Problem 1 with SQP from starting_point 1:
Initial guess x0:
[3 2]
Converged after 2 iters.
Problem 1, solution for starting_point 1: [2.33333333 2.        ]
Problem 1 with SQP from starting_point 2:
Initial guess x0:
[7 2]
Converged after 6 iters.
Problem 1, solution for starting_point 2: [2.33333333 2.        ]
Problem 2 with SQP from starting_point 1:
Initial guess x0:
[2.3 1.3]
Converged after 2 iters.
Problem 2, solution for starting_point 1: [2.66666667 0.66666667]
Problem 2 with SQP from starting_point 2:
Initial guess x0:
[1 3]
Converged after 4 iters.
Problem 2, solution for starting_point 2: [2.66666667 0.66666667]
Problem 3 with SQP from starting_point 1:
Initial guess x0:
[2 3]
Converged after 3 iters.
Problem 3, solution for starting_point 1: [1.07692308 3.69230769]
Problem 3 with SQP from starting_point 2:
Initial guess x0:
[ 2.  -0.5]
Converged after 6 iters.
Problem 3, solution for starting_point 2: [1.07692308 3.69230769]
Problem 4 with SQP from starting_point 1:
Initia